<img src="Archivos/miad4.png" width=800x>

## Interactividad usando `panel`

En este taller podrás poner en práctica los conceptos aprendidos acerca del uso del paquete `panel`. Usarás `panel` con el fin de crear dashboards como herramienta de apoyo al momento de resolver problemas específicos en un contexto de Analytics.

Instrucciones y condiciones generales: Esta es una actividad calificada y, por lo tanto, debe ser resuelta individualmente.

### Al desarrollar este taller podrás verificar tu progreso en:

1. Programar la funcionalidad de un dashboard en `panel`.
2. Agregar gráficos y widgets para hacer visualizaciones agradables en `panel`.
3. Personalizar dashobards para generar distintas visualizaciones interactivas en `panel`.

In [1]:
import panel as pn
import plotly.graph_objects as go
import pandas as pd
pn.extension('plotly')

## Pregunta 1

Con la base de datos *COVID-19.csv* crea un dashboard en `panel` que:

1. Permita crear un diagrama de barras horizontal que muestre los casos de COVID-19 de acuerdo a la selección de los siguentes criterios:<br />
    a. Casos o muertes.<br />
    b. Periodo: Histórico, Últimos 7 días y Últimas 24 horas.<br />
    Pista: utilice un control de tipo `RadioButtonGroup` para cada uno de los dos criterios.<br />
2. Contenga un diagrama de barras horizontal de las muertes por millón de personas.
3. Permita visualizar a tráves de un diagrama de barras horizontal los casos en cada una de las regiones de la Organización Munidal de la Salud.

Nota: a excpeción de las regiones, muestre los 10 mayores valores en cada gráfico.

| <center>Variable</center> |<center>Descripción</center> | 
| :-- | :-- | 
| Name | País |
| WHO Region | Región de la OMS |
| Cases - cumulative total | Total de casos hasta el momento |
| Cases - cumulative total per 1 million population | Número de casos acumulado por millón de personas |
| Cases - newly reported in last 7 days | Número de casos nuevos en los últimos 7 días |
| Cases - newly reported in last 24 hours | Número de casos nuevos en las últimas 24 horas |
| Deaths - cumulative total | Total de muertes hasta el momento |
| Deaths - cumulative total per 1 million population | Número de muertes acumulado por millón de personas |
| Deaths - newly reported in last 7 days | Número de muertes nuevas en los últimos 7 días |
| Deaths - newly reported in last 24 hours | Número de muertes nuevas en las últimas 24 horas |

In [75]:
#Cargamos la data:
    
datos = pd.read_csv('Archivos/COVID-19.csv')
datos = pd.DataFrame(datos)

datos.head()

,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 1 million population,Cases - newly reported in last 7 days,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 1 million population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 24 hours,Transmission Classification
0,United States of America,Americas,26547977,80204.73,871365,129961,455735,1376.83,22562,3549,Community transmission
1,India,South-East Asia,10826363,7845.17,80180,12059,154996,112.32,722,78,Clusters of cases
2,Brazil,Americas,9447165,44444.82,328652,50872,230034,1082.21,7368,1239,Community transmission
3,Russian Federation,Europe,3967281,27185.36,116842,16048,76661,525.31,3479,432,Clusters of cases
4,The United Kingdom,Europe,3929839,57888.79,133747,18262,112092,1651.18,6521,828,Community transmission


In [41]:
# PUNTO 1
titulo = '# casos de COVID-19'

In [42]:
#RadioButtonGroup.
cases_group = pn.widgets.RadioButtonGroup(
    name='Casos', options=['Número de casos', 'Número de muertes'], button_type='default')

period_group = pn.widgets.RadioButtonGroup(
    name='Periodo', options=['Histórico', 'Últimos 7 días', "Últimas 24 horas"], button_type='default')

cases_group
period_group

RadioButtonGroup(name='Periodo', options=['Histórico', ...], value='Histórico')

In [47]:
# Le decimos a panel que nuestro gráfico depende de cambios en el párametro seleccion en radiobottongroup
@pn.depends(cases_group.param.value, period_group.param.value)

# Definimos la función que se encarga de la interactividad del tablero de control.

def funcion_interactiva(cases_group,period_group): 
    
    #Cargamos los datos 
    df=datos
    
    #Filtros 
    if (cases_group == "Número de casos") and (period_group == "Histórico"):
        filtro = df[["Name","Cases - cumulative total"]]
        filtro = filtro.sort_values(by = ["Cases - cumulative total"],ascending = True)
    elif (cases_group == "Número de muertes") and (period_group == "Histórico"):
        filtro = df[["Name","Deaths - cumulative total"]]
        filtro = filtro.sort_values(by = ["Deaths - cumulative total"],ascending = True)
    elif (cases_group == "Número de casos") and (period_group == "Últimos 7 días"):
        filtro = df[["Name","Cases - newly reported in last 7 days"]]  
        filtro = filtro.sort_values(by = ["Cases - newly reported in last 7 days"],ascending = True)
    elif (cases_group == "Número de muertes") and (period_group == "Últimos 7 días"):
        filtro = df[["Name","Deaths - newly reported in last 7 days"]]
        filtro = filtro.sort_values(by = ["Deaths - newly reported in last 7 days"],ascending = True)
    elif (cases_group == "Número de casos") and (period_group == "Últimas 24 horas"):
        filtro = df[["Name","Cases - newly reported in last 24 hours"]]
        filtro = filtro.sort_values(by = ["Cases - newly reported in last 24 hours"],ascending = True)
    elif (cases_group == "Número de muertes") and (period_group == "Últimas 24 horas"):
        filtro = df[["Name","Deaths - newly reported in last 24 hours"]] 
        filtro = filtro.sort_values(by = ["Deaths - newly reported in last 24 hours"],ascending = True)
        
    fig = go.Figure(go.Bar(
                        x = filtro.iloc[:, 1], 
                        y = filtro["Name"], 
                        text = filtro.iloc[:, 1], 
                        textposition = "auto", 
                        texttemplate = "%{text:.0f}",
                        marker_color = "rgb(126, 126, 126)", orientation = "h"))

                    
    fig.update_layout(barmode = "stack", 
                      yaxis_title = "País",
                      xaxis_title = "Número de casos/muertes",
                        title = "Casos/Muertes",
                        xaxis = dict(showgrid=False),yaxis = dict(showgrid=False),plot_bgcolor = "white",showlegend = False)
    
    return fig

In [48]:
tablero_control = pn.Row(pn.Column(cases_group, period_group), funcion_interactiva)

In [49]:
tablero_control.servable()

Row
    [0] Column
        [0] RadioButtonGroup(name='Casos', options=['Número de casos', ...], value='Número de casos')
        [1] RadioButtonGroup(name='Periodo', options=['Histórico', ...], value='Histórico')
    [1] ParamFunction(function)

In [78]:
#2. Contenga un diagrama de barras horizontal de las muertes por millón de personas.

titulo = 'Muertes por millón de personas'

def muertes_por_millon():
    df = datos[["Name", "Deaths - cumulative total per 1 million population"]]
    df = df.sort_values(by = "Deaths - cumulative total per 1 million population", inplace = False, ascending = False).head(10) 
    
    fig = go.Figure(go.Bar(
        x=df["Deaths - cumulative total per 1 million population"],
        y=df["Name"],
        text=df["Deaths - cumulative total per 1 million population"],
        textposition='outside',                                # Posición de la etiqueta de datos. En este caso a la derecha.
        texttemplate='%{text:,.2f}',                            # Formato de texto.
        marker_color='rgb(30, 118, 255)',                      # Color de las barras.
        orientation='h'                                        # Orientación de las barras. En este caso horizontal.
    ))

    fig.update_layout(barmode = "stack", 
                      yaxis_title = "País",
                      xaxis_title = "Muertes por millon",
                        title = "Muertes por millon",
                        xaxis = dict(showgrid=False),yaxis = dict(showgrid=False),plot_bgcolor = "white",showlegend = False)
    
    return fig


In [85]:
#3. Permita visualizar a tráves de un diagrama de barras horizontal los casos en cada una de las regiones de la Organización Munidal de la Salud

titulo = 'Casos por region'

def casos_region():
    df = datos[["WHO Region", "Cases - cumulative total"]]
    df=df.groupby('WHO Region').sum().reset_index()
    df = df.sort_values(by = "Cases - cumulative total", inplace = False, ascending = False).head(10) 
    
    fig = go.Figure(go.Bar(
        x=df["Cases - cumulative total"],
        y=df["WHO Region"],
        text=df["Cases - cumulative total"],
        textposition='outside',                                # Posición de la etiqueta de datos. En este caso a la derecha.
        texttemplate='%{text:,.2f}',                            # Formato de texto.
        marker_color='rgb(30, 118, 255)',                      # Color de las barras.
        orientation='h'                                        # Orientación de las barras. En este caso horizontal.
    ))

    fig.update_layout(barmode = "stack", 
                      yaxis_title = "País",
                      xaxis_title = "Casos",
                        title = "Casos por region",
                        xaxis = dict(showgrid=False),yaxis = dict(showgrid=False),plot_bgcolor = "white",showlegend = False)
    
    return fig

In [86]:
tablero_control = pn.Row(pn.Column(titulo, cases_group, period_group),
                         pn.Column(funcion_interactiva, muertes_por_millon, casos_region))

In [87]:

tablero_control.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] RadioButtonGroup(name='Casos', options=['Número de casos', ...], value='Número de casos')
        [2] RadioButtonGroup(name='Periodo', options=['Histórico', ...], value='Histórico')
    [1] Column
        [0] ParamFunction(function)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive00304')
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive00310')

## Créditos

**Autor(es)**: Juan David Reyes Jaimes, Diego Alejandro Cely Gomez

**Última actualización:** 26/08/2021